In [13]:
import pandas as pd
import numpy as np

In [14]:
# edit charter_to_district data for calculating multipliers
df_mult = pd.read_csv('./data/finalized/charter_to_district.csv')
print(df_mult.dtypes)
print(df_mult.shape)

# filter to just fy21, since we're missing many years in the data
df_mult = df_mult[df_mult.fy.eq('fy21')]
print(df_mult.shape)
del df_mult['year']
del df_mult['fy']

fy                            object
year                           int64
charter_lea_code               int64
sending_lea_code               int64
enrolled_n                     int64
physical_charter_location     object
chartered_to_serve           float64
dtype: object
(5588, 7)
(1029, 7)


In [15]:
# create enrollment remainders for all district schools
df_enrollment = pd.read_csv('./data/finalized/district_enrollment.csv', dtype={'num': np.int64, 'district_id': np.int64})
df_enrollment = df_enrollment.rename(columns={'district_id': 'sending_lea_code'})
df_sending = df_mult[['sending_lea_code', 'enrolled_n']].groupby('sending_lea_code',as_index=False)['enrolled_n'].sum()

df_enrollment = df_enrollment.merge(df_sending, on='sending_lea_code')
del df_sending
df_enrollment['diff'] = df_enrollment['num'] - df_enrollment['enrolled_n']
del df_enrollment['num']
del df_enrollment['enrolled_n']
df_enrollment = df_enrollment.rename(columns={'diff': 'enrolled_n'})
print(df_enrollment.head(5))

   sending_lea_code  enrolled_n
0               780        3532
1               778        1158
2               775        6535
3               774         355
4               773        2129


In [16]:
# export
df_mult.to_csv('./data/finalized/charter_to_district_edited.csv', index=False)
df_enrollment.to_csv('./data/finalized/district_remainders.csv', index=False)